# Progetto Introduzione all'intelligenza artificiale

Il progetto consiste nella realizzazione di una applicazione di Intelligenza Artificiale completa degli
aspetti di gestione di: sensori per l’acquisizione dei dati dall’esterno relativi a stati e obiettivi,
ragionamento/ricerca della soluzione per i goal acquisiti, esecutori per la realizzazione delle azioni
che conducono alla soluzione.

## Progetto Uniform Coloring
Uniform Coloring è un dominio in cui si hanno a disposizione alcune celle da colorare, e vari colori a disposizione. 
Per semplicità immaginiamo una griglia rettangolare in cui è possibile spostare una testina colorante fra le celle attigue secondo le 4 direzioni cardinali (N,S,E,W), senza uscire dalla griglia. 

Tutte le celle hanno un colore di partenza (B=blu, Y=yellow, G=green) ad eccezione di quella in cui si trova la testina indicata con T. La testina può colorare la cella in cui si trova con uno qualsiasi dei colori disponibili a differenti costi (cost(B)=1, cost(Y)=2, cost(G)=3), mentre gli spostamenti hanno tutti costo uniforme pari a 1. 

>L’obiettivo è colorare tutte le celle dello stesso colore (non importa quale) e riportare la testina nella sua posizione di partenza.

La codifica di tutto il dominio (topologia della griglia, definizione delle azioni etc.) è parte dell’esercizio. Partendo dalla posizione iniziale della testina e combinando azioni di spostamento e colorazione, si chiede di trovare la sequenza di azioni dell’agente per raggiungere l’obiettivo.

La posizione iniziale della testina, la struttura della griglia e la colorazione iniziale delle celle sono passati al sistema tramite un’immagine.

### Inizializzazione delle librerie e moduli

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import datetime
from emnist import list_datasets
from tensorflow import keras    
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report



list_datasets()

'''
# print some images from the dataset to test
import matplotlib.pyplot as plt
from emnist import extract_training_samples
images, labels = extract_training_samples('letters')
for i in range(100):
    plt.subplot(10, 10, i+1)
    plt.axis('off')
    plt.imshow(images[i], cmap='gray')
plt.show()
'''


"\n# print some images from the dataset to test\nimport matplotlib.pyplot as plt\nfrom emnist import extract_training_samples\nimages, labels = extract_training_samples('letters')\nfor i in range(100):\n    plt.subplot(10, 10, i+1)\n    plt.axis('off')\n    plt.imshow(images[i], cmap='gray')\nplt.show()\n"

## 1. Descrizione formale del dominio e vincoli su cui l'agente può operare

Descrizione formale del dominio e dei vincoli che le azioni eseguibili dall’agente (la testina) nella griglia devono rispettare (esempio vincoli: v1=”l’agente può compiere un solo passo alla volta”, v2=”l’agente si può muovere solo fra celle adiacenti”, etc.). La descrizione sarà un testo che descrive le regole da rispettare e le assunzioni desunte dall’analisi del dominio.

### Descrizione del dominio e vincoli 

**Elementi del dominio:**
- Celle: Presenti in una griglia rettangolare nella quale è possibile spostare una testina colorante. Ogni cella ha un colore di partenza ed è possibile ricolorarle con i tre colori disponibili (yellow, blue, green).
- Testina colorante: È l'agente che, nella griglia fornita in input come immagine, può spostarsi tra le celle e cambiarne il colore. Una delle celle rappresenta la posizione iniziale della testina prima di muoversi e sulla quale dovrà ritornare dopo aver svolto le azioni richieste.

**Relazioni:**
- Ad ogni cella dell'immagine è associata un'etichetta rappresentante uno dei colori disponibili (Y, B, G).
- La testina inizialmente dovrà essere posizionata sempre sulla cella con etichetta T.

**Regole:**
- La testina può spostarsi nelle sole direzioni nord, sud, est, ovest.
- La testina può cambiare colore nella cella in cui è posizionata. Colorare le celle ha un costo che varia in base al colore (cost(B) = 1, cost(Y) = 2, cost(G) = 3).
- Il passaggio da una cella all'altra ha sempre costo 1.
- **GOAL**: Colorare tutte le celle dello stesso colore. La testina dovrà trovare un modo per farlo nella maniera più efficiente possibile, sia in termini di colori che di numero di passi effettuati per muoversi tra le celle.

**Vincoli:**
- v0="La griglia può essere rettangolare e quadrata".
- v1=”L’agente può compiere un solo passo alla volta”.
- v2=”L’agente si può muovere solo fra celle adiacenti”.
- v3="Nella griglia non esistono celle vuote, tutte devono essere colorate in partenza".
- v4="Le celle devono essere tutte raggiungibili dalla posizione iniziale della testina".
- v5="Il costo delle azioni di movimento è uniforme (1), mentre il costo della colorazione dipende dal colore scelto".
- v6="L'agente non può colorare la posizione di partenza (quindi bisogna trovare un modo per evitarlo)".
- v7="Dopo che l'agente ha colorato tutte le celle, deve ritornare alla posizione di partenza".

**Esempi di problemi, con possibili soluzioni e costi**:
che sono costituite da un array bidimsionale, verranno fornite in un immagine come input al programma  

I vincoli applicati sono i seguenti:
- v1=”l’agente può compiere un solo passo alla volta”
- v2=”l’agente si può muovere solo fra celle adiacenti”
- v3="dopo che l'agente ha colorato tutte le testine, deve ritornare alla posizione di partenza"
- v4="l'agente non puo colorare la posizione di partenza (quindi bisogna trovare un modo per evitarlo)"

## 2 Implementazione delle classi per la ricerca nello spazio degli stati di Smart Vacuum.
Utilizzando le classi di AIMA-python si implementi quindi un dominio UniformColoring come classe derivata da Problem, scegliendo e definendo una rappresentazione per gli stati e ridefinendo opportuni metodi actions e result e tutti gli eventuali metodi aggiuntivi , es. goal_test, che si rendessero necessari.
Si descriva e si implementi almeno una euristica definenendone le caratteristiche di consistenza e ammissibilità rispetto al dominio dato. L’euristica definita mantiene le stesse proprietà nel caso in cui le azioni di spostamento costassero 0 o le colorazioni avessero tutte lo stesso costo?

## 3 Acquisizione e classificazione degli input, stato iniziale e goal. 
Si realizzi un programma che, passata in input un’immagine contenente la configurazione della griglia e la posizione iniziale dell’agente:
a) interpreti l’immagine individuando la configurazione della griglia e, attraverso un metodo di classificazione, la posizione iniziale dell’agente e la colorazione iniziale delle celle. Non ci sono vincoli sul metodo/modello utilizzato per la classificazione. 
Si consiglia di utilizzare il dataseteMNIST/MNIST per la classficazione di lettere e cifre scritte a mano, visto a lezione e facilmente reperibile su Web. Assumiamo che le lettere siano solo maiuscole;
b) traduca i dati risultanti dall’analisi delle immagini negli stati stato_iniziale e stato_goal del problema, secondo la rappresentazione definita per il punto 2;
c) invochi il solutore (vedi punto 2), tramite una tecnica di ricerca informata e almeno una non informata, della classe UniformColoring e produca, se esiste, la soluzione del problema, ovvero la sequenza azioni da eseguire per raggiungere lo stato goal. E’ interessante mostrare come algoritmi diversi possano portare a soluzioni diverse nel caso in cui ottimizzino rispetto al costo della soluzione oppure rispetto la sua lunghezza.

In [2]:
# Import the necessary functions from the emnist library
from emnist import extract_training_samples
from emnist import extract_test_samples

# Define the labels for the EMNIST dataset
LABELS = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 
          'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
          'a', 'b', 'd', 'e', 'f', 'g', 'h', 'n', 'q', 'r', 't']

# Define a function to remodulate the labels
def remodulate(y):
    """
    Remodulates the input array 'y' based on the labels.

    Parameters:
    - y (numpy.ndarray): The input array to be remodulated.

    Returns:
    - numpy.ndarray: The remodulated array.
    """
    # Replace the labels 'T', 'B', 'Y', 'G' with 0, 1, 2, 3 respectively
    # Define a dictionary to map the labels to their new values
    label_map = {"T": 0, "B": 1, "Y": 2, "G": 3}

    # Loop through the label_map to replace the labels with their new values
    for label, value in label_map.items():
        y = np.where(y == LABELS.index(label), value, y)

    return y

# Define a function to filter the dataset based on specific label conditions
def filterDataset(X_data, y_data):
    """
    Filters the dataset based on lables we need to use (T, B, G, Y).

    Args:
        X_data (ndarray): Input data array of shape (n_samples, 28, 28).
        y_data (ndarray): Target labels array of shape (n_samples,).

    Returns:
        tuple: A tuple containing the filtered input data array and target labels array.

    Raises:
        AssertionError: If the dtype of y_data is not np.uint8.
    """
    # Assert that the dtype of y_data is np.uint8
    assert y_data.dtype == np.uint8
    # Define the classes
    classes = LABELS
    # Initialize the new_data_size
    new_data_size = 0
    # Initialize the new_data_index
    new_data_index = 0

    # Loop through the y_data to count the number of records with labels 'B', 'Y', 'G', 'T'
    for recordIndex in range(0, y_data.shape[0]):
        currentLabel = classes[y_data[recordIndex]]
        if currentLabel == 'B' or currentLabel == 'Y' or currentLabel == 'G' or currentLabel == 'T':
            new_data_size += 1
    
    # Initialize the new_X_data and new_y_data with zeros
    new_X_data = np.zeros((new_data_size, 28, 28), dtype = X_data.dtype)
    new_y_data = np.zeros((new_data_size,), dtype = np.uint8)

    # Loop through the y_data to filter the records with labels 'B', 'Y', 'G', 'T'
    for recordIndex in range(0, y_data.shape[0]):
        currentLabel = classes[y_data[recordIndex]]

        if currentLabel != 'B' and currentLabel != 'Y' and currentLabel != 'G' and currentLabel != 'T':
            continue
        
        new_X_data[new_data_index] = X_data[recordIndex]
        new_y_data[new_data_index] = y_data[recordIndex]
        new_data_index += 1
    
    # Assert that the new_data_index is equal to the shape of new_X_data
    assert new_data_index == new_X_data.shape[0]
    return (new_X_data, new_y_data)

# Extract the training and test samples from the EMNIST dataset
X_train, y_train = extract_training_samples('balanced')
X_test, y_test = extract_test_samples('balanced')

# Filter the training and test datasets
(X_train, y_train_library) = filterDataset(X_train, y_train)
(X_test, y_test) = filterDataset(X_test, y_test)

# Remodulate the labels of the training and test datasets
y_train_library = remodulate(y_train_library)
y_test = remodulate(y_test)

# Normalize the training and test datasets
X_train = X_train.astype("float32") / 255
X_test = X_test.astype("float32") / 255

KeyError: "There is no item named 'gzip/emnist-balanced-train-images-idx3-ubyte.gz' in the archive"

### Reshape
Il seguente codice sta ridisponendo i dati di addestramento e test _da 3D_ (numero di campioni, larghezza dell'immagine, altezza dell'immagine) _a 2D_ (numero di campioni, larghezza dell'immagine * altezza dell'immagine).
Molti modelli di reti neurali si aspettano di ricevere dati in un formato bidimensionale, quindi se i tuoi dati sono in un formato diverso (ad esempio, immagini rappresentate come array tridimensionali), potrebbe essere necessario eseguire il reshape per adattarli alla forma richiesta dal modello..

In [ ]:
# Print the shape of the training data
print(X_train.shape)

# Reshape the training data from 3D to 2D. The new shape is (number of samples, image width * image height)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1]* X_train.shape[2])

# Reshape the test data from 3D to 2D. The new shape is (number of samples, image width * image height)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1]* X_test.shape[2])

# Print the new shape of the training data
print(X_train.shape)

# Print the shape of the test data
print(X_test.shape)

### Creazione della rete neurale per il modello 

In [ ]:
# Initialize a Sequential model
seq_lett_model = keras.Sequential()

# Add an Input layer to the model. This layer specifies the shape of the input data (784 features).
seq_lett_model.add(keras.Input(shape=(784,)))

# Add a Dense layer to the model with 256 neurons and a ReLU activation function.
# Dense layers are fully connected layers, meaning all the neurons in a layer are connected to those in the next layer.
seq_lett_model.add(keras.layers.Dense(256, activation='relu'))

# Add another Dense layer to the model with 128 neurons and a ReLU activation function.
seq_lett_model.add(keras.layers.Dense(128, activation='relu'))

# Add a final Dense layer with 4 neurons (because we have 4 letters to detect) and a softmax activation function.
# The softmax function is often used in the final layer of a neural network-based classifier.
seq_lett_model.add(keras.layers.Dense(4, activation='softmax'))

# Print a summary of the model, including the number of parameters and the shape of the output at each layer.
seq_lett_model.summary()

### Training

In [ ]:
# Set the batch size. This is the number of samples that will be passed through the network at once.
batch_size = 512

# Set the number of epochs to 7. An epoch is one complete pass through the entire training dataset.
epochs = 10

# Compile the model. 
# We use the sparse_categorical_crossentropy loss function, which is suitable for multi-class classification problems.
# The optimizer is set to 'adam', which is a popular choice due to its efficiency and good performance on a wide range of problems.
# We also specify that we want to evaluate the model's accuracy during training.
seq_lett_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Fit the model to the training data. 
# We also specify a validation split of 0.1, meaning that 10% of the training data will be used as validation data.
# The model's performance is evaluated on this validation data at the end of each epoch.
training_operation = seq_lett_model.fit(X_train, y_train_library, batch_size=batch_size, epochs=epochs, validation_split=0.1)

# Save the trained model to a file so that it can be loaded later for making predictions or continuing training.
seq_lett_model.save('seq_lett_model.keras')

In [ ]:
# Plot training & validation accuracy values
plt.plot(training_operation.history['accuracy'])
plt.plot(training_operation.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(training_operation.history['loss'])
plt.plot(training_operation.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

evaluating the performance of a trained model on test data. It prints the loss and accuracy that the model achieved on the test data. The loss is a measure of how well the model can estimate the target variables, while the accuracy is the proportion of correct predictions that the model made.

In [ ]:
# Evaluate the model's performance on the test data. 
# The evaluate function returns the loss value and metrics values for the model in test mode.
# We set verbose=0 to avoid logging the detailed output during the evaluation.
loss, accuracy =  seq_lett_model.evaluate(X_test, y_test, verbose=0)

# Print the loss value that the model achieved on the test data.
# The loss value represents how well the model can estimate the target variables. Lower values are better.
print("Test loss:", loss)

# Print the accuracy that the model achieved on the test data.
# The accuracy is the proportion of correct predictions that the model made. Higher values are better.
print("Test accuracy:", accuracy)

### confusion matrix

In [ ]:
# Use the trained model to make predictions on the test data.
# The predict function returns the output of the last layer in the model, which in this case is the output of the softmax layer.
y_pred = seq_lett_model.predict(X_test)

# The output of the softmax layer is a vector of probabilities for each class. 
# We use the argmax function to find the index of the maximum probability, which gives us the predicted class.
y_pred = np.argmax(y_pred, axis = 1)

# Compute the confusion matrix to evaluate the accuracy of the classification.
# The confusion matrix is a table that is often used to describe the performance of a classification model.
# Each row of the matrix represents the instances in a predicted class, while each column represents the instances in an actual class.
# The 'normalize' parameter is set to 'true', which means the confusion matrix will be normalized by row (i.e., by the number of samples in each class).
confusionMatrix = confusion_matrix(y_test, y_pred, normalize = 'true')

# Create a ConfusionMatrixDisplay object from the confusion matrix.
# The display_labels parameter is set to the names of the classes.
disp = ConfusionMatrixDisplay(confusion_matrix = confusionMatrix, display_labels = ['T','B','Y','G'])

# Plot the confusion matrix.
disp.plot()

# Display the plot.
plt.show()

##### Precision, Recall e F1

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

### Extraction from the grid

In [ ]:
!find  './manipulated_grids/' -name 'ROI_*' -exec rm {} \;

font = cv2.FONT_HERSHEY_COMPLEX 

# Open the webcam
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()

    # If the frame was read correctly, ret is True
    if ret:
        # Display the frame
        cv2.imshow('Input', frame)

        # If the 'q' key is pressed, break from the loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Get the current date and time
now = datetime.datetime.now()

# Format the date and time as a string
# This will give a string like '2022-03-15_14-30-20' for March 15, 2022 at 2:30:20 PM
timestamp = now.strftime('%Y-%m-%d_%H-%M-%S')

# Use the timestamp in the filename
file_name = './manipulated_grids/grid_{}.png'.format(timestamp)
cv2.imwrite(file_name, frame)

# Release the webcam and destroy all windows
cap.release()
cv2.destroyAllWindows()

# Load the image that was just saved
image  = cv2.imread('./manipulated_grids/grid_2024-03-17_16-12-12.png')
original = image.copy()
ROI_number = 0

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Save the grayscale image
cv2.imwrite('./manipulated_grids/gray.png', gray)

# Apply Gaussian blur to the grayscale image
blur = cv2.GaussianBlur(gray, (5,5), 0)

# Save the blurred image
cv2.imwrite('./manipulated_grids/blur.png', blur)

# Apply adaptive thresholding to the blurred image
thresh = cv2.adaptiveThreshold(blur, 255, 1, 1, 11, 2)

# Save the thresholded image
cv2.imwrite('./manipulated_grids/tresh.png', thresh)

# Find contours in the thresholded image
contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Initialize variables for the contour with the largest area
max_area = 0
c = 0

# Loop through the contours
for i in contours:
    # Calculate the area of the contour
    area = cv2.contourArea(i)

    # If the area is larger than 70*60
    if area > 70*60:
        # If the area is larger than the current max_area
        if area > max_area:
            # Update max_area and best_cnt
            max_area = area
            best_cnt = i

        # Draw the contour on the image
        image = cv2.drawContours(image, contours, c, (0, 255, 0), 3)
        plt.imshow(image)
    c+=1

# Create a binary mask where the largest contour is filled with white
mask = np.zeros((gray.shape),np.uint8)
cv2.drawContours(mask,[best_cnt],0,255,-1)
cv2.drawContours(mask,[best_cnt],0,0,2)

# Apply the mask to the grayscale image
out = np.zeros_like(gray)
out[mask == 255] = gray[mask == 255]

# Initialize variables
c = 0
n = 0
yt = 0
l = []

# Loop through the contours again
for i in contours:
    # Calculate the area of the contour
    area = cv2.contourArea(i)

    # If the area is larger than 70*60
    if area > 70*60:
        # Get the bounding rectangle for the contour
        x,y,w,h = cv2.boundingRect(i)

        # Draw a green rectangle around the contour on the image
        plt.imshow(cv2.rectangle(image, (x, y), (x + w, y + h), (36,12,12), 2))

        # Extract the region of interest (ROI) from the original image
        ROI = original[y:y+h, x:x+w]

        # If this is not the first ROI
        if ROI_number != 0:
            # If the y-coordinate is the same as the previous ROI
            if yt == y:
                # Remove the border from the ROI and save it as an image
                ROI = ROI[y:y+h, x:x+w]
                cv2.imwrite('./manipulated_grids/ROI_{}.png'.format(ROI_number), ROI)
                n+=1
            else:
                # Remove the border from the ROI, update yt, and save the ROI as an image
                ROI = ROI[y:y+h, x:x+w]
                yt = y
                cv2.imwrite('./manipulated_grids/ROI_{}.png'.format(ROI_number), ROI)
                n = 1

        # Increment ROI_number
        ROI_number += 1        

        # Draw the contour on the image
        plt.imshow(cv2.drawContours(image, contours, c, (0, 255, 0), 3))
    c+=1

cell calssifier

In [ ]:

l=[]
for i in range(1, ROI_number):
  im = cv2.imread("./manipulated_grids/ROI_"+str(i)+".png")
  im = cv2.bitwise_not(im)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  (thresh, im) = cv2.threshold(im, 127, 255, cv2.THRESH_BINARY)
  
  #im = im[3:67,3:67]  #rimuovi bordo

  im = cv2.resize(im, dsize=(28, 28), interpolation=cv2.INTER_LINEAR)


  im = np.reshape(im, -1)
  im = im.astype("float32")/255
  im = tf.expand_dims(im, 0)

  prediction = seq_lett_model.predict(im)
  max = np.where(prediction == np.amax(prediction))
  l.append(int(max[1]))


if n < len(l):
    nrow= len(l)/n
else:
    nrow=n/len(l)

nrow = int(nrow)

mat = np.array(list(reversed(l)))

grid=mat.reshape(nrow, n)
show=np.copy(grid)
show = np.where(show==0,'T',show)
show = np.where(show=='1','B',show)
show = np.where(show=='2','Y',show)
show = np.where(show=='3','G',show)
print(show)

## 4 Esecuzione. 
Si implementi un simulatore dell’esecuzione del piano di azioni calcolato, anche semplicemente attraverso una sequenza di immagini.